Mon 9/17/18 Creating SQL database from Etsy data

In [1]:
import pandas as pd
import json
from glob import glob
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
pd.set_option('display.max_columns', 50)

In [2]:
cd ~/Documents/Insight/Etsy/180912_1st_Exploratory_Data_Analysis/

/home/mc/Documents/Insight/Etsy/180912_1st_Exploratory_Data_Analysis


In [3]:
with open('listings/0001.json', 'r') as listings_file:
    etsy_df = pd.read_json(listings_file)

Create dataframes for each table in the dataset
    1. listings table
    2. tags table
    3. materials table
    4. categories table
    5. category ids table
    6. styles table

In [4]:
# 1. main product table
column_names = ['listing_id','user_id','category_id','shop_section_id','price','currency_code','views','num_favorers','title','url','quantity','featured_rank','recipient','occasion','when_made','who_made','has_variations','is_customizable','is_digital','is_private','is_supply','language','original_creation_tsz','creation_tsz','ending_tsz','last_modified_tsz','state_tsz','processing_min','processing_max','state','non_taxable']
listings = etsy_df[column_names].copy(deep=True)

# 2. tags table
tags = etsy_df[['listing_id','tags']].copy(deep=True).set_index('listing_id').tags.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index().rename(index=str, columns={0: "tag"})

# 3. materials table
materials = etsy_df[['listing_id','materials']].copy(deep=True).set_index('listing_id').materials.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index().rename(index=str, columns={0: "material"})

# 4. categories table
categories = etsy_df[['listing_id','category_path']].copy(deep=True).set_index('listing_id').category_path.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index().rename(index=str, columns={0: "category"})

# 5. category ids table
category_ids = etsy_df[['listing_id','category_path_ids']].copy(deep=True).set_index('listing_id').category_path_ids.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(int).reset_index().rename(index=str, columns={0: "category_id"})

# 6. styles table
# NOTE: "style" is a keyword in pandas, so the the column name has to be changed to "styles" first
styles = etsy_df[['listing_id','style']].copy(deep=True)
styles = styles.rename(index=str, columns={"style": "styles"})
styles = styles.set_index('listing_id').styles.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index().rename(index=str, columns={0: "style"})

table_list = [listings, tags, materials, categories, category_ids, styles]

In [5]:
listings

,listing_id,user_id,category_id,shop_section_id,price,currency_code,views,num_favorers,title,url,quantity,featured_rank,recipient,occasion,when_made,who_made,has_variations,is_customizable,is_digital,is_private,is_supply,language,original_creation_tsz,creation_tsz,ending_tsz,last_modified_tsz,state_tsz,processing_min,processing_max,state,non_taxable
0,101000405,20198008,68902992,11235567.0,14.50,USD,872,33,Little Reversible Dress in Brown Flower and Pu...,https://www.etsy.com/listing/101000405/little-...,1,NaN,None,None,2010_2014,i_did,True,True,False,False,false,en-US,1338408908,1406467475,1417098275,1407025471,1406225982,3.0,5.0,active,False
1,101000518,14657489,68907858,11797420.0,12.00,USD,487,18,Pollux Pendant -Beading Pattern/Tutorial - PDF...,https://www.etsy.com/listing/101000518/pollux-...,1,NaN,None,None,2010_2014,i_did,False,False,True,False,true,en-US,1338409003,1405107171,1415737971,1405107171,1405100435,NaN,NaN,active,False
2,101000649,20823487,69153339,13025088.0,15.99,USD,1000,87,Pet Collar Tags Personalized - Dog Bow Tie Las...,https://www.etsy.com/listing/101000649/pet-col...,5,NaN,pets,None,made_to_order,i_did,False,False,False,False,false,en-US,1338409086,1412182158,1422812958,1412182570,1368754108,NaN,NaN,active,False
3,101000651,19354077,68889592,11668909.0,5.00,USD,9,0,Red Rose,https://www.etsy.com/listing/101000651/red-ros...,1,NaN,women,None,2010_2014,i_did,False,True,False,False,false,en-US,1338409089,1413312965,1423943765,1413312965,1413312643,1.0,2.0,active,False
4,101000721,7116046,68902452,6381199.0,10.00,USD,978,100,Brachiosaurus,https://www.etsy.com/listing/101000721/brachio...,7,NaN,children,None,2010_2014,i_did,False,True,False,False,false,en-US,1338409140,1411676367,1422220767,1411676367,1389997787,NaN,NaN,active,False
5,101000734,8466169,68925824,6560153.0,30.00,USD,157,7,Speak your truth Soulful Yoga Plow Pose artwor...,https://www.etsy.com/listing/101000734/speak-y...,7,NaN,unisex_adults,None,2010_2014,i_did,False,True,False,False,false,en-US,1338409149,1409529989,1420074389,1411734838,1400031030,NaN,NaN,active,False
6,101000758,21245871,68918698,11492360.0,18.00,USD,188,14,Personalized Name and Monogram Vinyl Decal- Gi...,https://www.etsy.com/listing/101000758/persona...,11,NaN,None,None,made_to_order,i_did,True,True,False,False,false,en-US,1338409164,1412446385,1423077185,1413548399,1411594946,1.0,3.0,active,False
7,101001157,16360331,68897044,14356960.0,9.95,USD,136,293,Felted Bird brooch with Beads and Sequins - F...,https://www.etsy.com/listing/101001157/felted-...,1,NaN,women,july_4th,2010_2014,i_did,False,True,False,False,false,en-US,1338409430,1403605766,1414146566,1407590177,1403373635,NaN,NaN,active,False
8,101001245,5312557,69173677,5732269.0,19.99,USD,519,50,Owl with Letters Custom Return Address Rubber ...,https://www.etsy.com/listing/101001245/owl-wit...,4,NaN,None,None,made_to_order,i_did,True,True,False,False,false,en-US,1338409493,1411855712,1422400112,1411855712,1401945619,NaN,NaN,active,False
9,101001303,8466169,68925824,6560153.0,30.00,USD,110,6,The Universe will Rally Soulful Yoga Plow Pose...,https://www.etsy.com/listing/101001303/the-uni...,12,NaN,unisex_adults,None,2010_2014,i_did,False,True,False,False,false,en-US,1338409520,1412867523,1423498323,1412867523,1392496833,NaN,NaN,active,False


In [6]:
to_int_list = ['user_id','category_id','shop_section_id','views','num_favorers','quantity','featured_rank','has_variations','is_customizable','is_digital','is_private','original_creation_tsz','ending_tsz','last_modified_tsz','state_tsz','processing_min','processing_max','non_taxable']
listings[to_int_list] = listings[to_int_list].astype(float)

In [8]:
listings.

listing_id                 int64
user_id                  float64
category_id              float64
shop_section_id          float64
price                    float64
currency_code             object
views                    float64
num_favorers             float64
title                     object
url                       object
quantity                 float64
featured_rank            float64
recipient                 object
occasion                  object
when_made                 object
who_made                  object
has_variations           float64
is_customizable          float64
is_digital               float64
is_private               float64
is_supply                 object
language                  object
original_creation_tsz    float64
creation_tsz               int64
ending_tsz               float64
last_modified_tsz        float64
state_tsz                float64
processing_min           float64
processing_max           float64
state                     object
non_taxabl

Create Etsy Database

In [2]:
# Define a database name (etsy)
# Set your postgres username/password, and connection specifics
username = 'postgres'
password = 'jlmlcook'
host     = 'localhost'
port     = '5432'
db_name  = 'etsy'

In [ ]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine( 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, db_name) )
print(engine.url)

In [ ]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

In [ ]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
listings.to_sql("listings", engine, index=False, if_exists='append')
tags.to_sql("tags", engine, index=False, if_exists='append')
materials.to_sql("materials", engine, index=False, if_exists='append')
categories.to_sql("categories", engine, index=False, if_exists='append')
category_ids.to_sql("category_ids", engine, index=False, if_exists='append')
styles.to_sql("styles", engine, index=False, if_exists='append')

with engine.connect() as con:
    con.execute('ALTER TABLE listings ADD PRIMARY KEY (listing_id);')

In [3]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(user = username, password = password, host = host, port = port, database = db_name)

In [5]:
# query:
sql_query = """
SELECT COUNT(listing_id) FROM listings;
"""
test = pd.read_sql_query(sql_query,con)
print(pd.read_sql_query(sql_query,con))

   count
0  20000


In [13]:
print(test['count'][0])

20000


In [ ]:
# query:
sql_query = """
SELECT * FROM products_table;
"""

products2 = pd.read_sql_query(sql_query,con)

In [ ]:
file_list = sorted(glob('listings/*.json'))
file_list = ['listings/0000.json']
for i in range(len(file_list)):
    print(file_list[i])